# Voxelization Procedure

In this notebook I will illustrate the procedure for voxelization of the mesh files that will be the preprocessing fo the data for our VoxelNet model.

The _ModelNet_ dataset comprises a number of samples for 10 different labeled objects, that are originally provided in `.off` files; these files are a collection of edges and faces. 

To process these files, we will be using <a href=https://docs.pyvista.org/>PyVista</a>, which is a high-level API that interfaces with <a href=https://vtk.org/>VTK</a>. Unfortunately, the `.off` protocol is not supported by VTK so we first had to convert the dataset to `.ply` format, which is still a list of edges and faces.

***

We begin by import `pyvista` and setting the rendering backend to `ipygany`; this backend streams mesh data directly to the client and avoid server-side rendering which on our VM won't work. 

In [1]:
import pyvista as pv
import numpy as np

# necessary: streams mesh directly to client and rendering happens there
pv.set_jupyter_backend('ipygany')

files = !ls *.ply
print('Local files:')
print(files,'\n\n')

# load mesh
file = files[0]
print(file,'\n')
mesh=pv.read(file)

mesh

Local files:
['desk_0001.ply', 'monitor_0001.ply'] 


desk_0001.ply 



PolyData,Information
N Cells,8748
N Points,12036
N Strips,0
X Bounds,"-1.325e+01, 1.325e+01"
Y Bounds,"-4.625e+01, 4.625e+01"
Z Bounds,"-1.400e+01, 1.400e+01"
N Arrays,0


Now, the _ModelNet_ dataset is already manually aligned, so we will rotate the mesh around the origin by a random angle. Then, we want to translate the mesh so that its outline fits perfectly into the region $[-1,1]^3$.

In [2]:
# rotate it randomly
x_deg,y_deg,z_deg=360*np.random.rand(3)

#pivot=mesh.outline().center_of_mass() is this needed?
pivot = np.zeros(3)

mesh.rotate_x(x_deg,pivot,inplace=True)
mesh.rotate_y(y_deg,pivot,inplace=True)
mesh.rotate_z(z_deg,pivot,inplace=True)

pivot=mesh.outline().center_of_mass()
mesh.translate(-pivot,inplace=True)

# scale so that it fits into [-1,1]^3 region
scaling_factor = 1/max(abs(np.array(mesh.bounds)))
mesh.scale(scaling_factor,inplace=True)

p = pv.Plotter()
p.add_mesh(mesh)
p.view_isometric()
p.show()

Scene(background_color='#4c4c4c', camera={'position': [1.7667358235657487, 1.7667358235657487, 1.7667358235657…

It is time to define the grid: we will be creating a $N\times N\times N$ grid that spans the $[-1,1]^3$ interval.

In [3]:
# the grid
N_linear_cells=30
resolution = N_linear_cells*np.ones(3,dtype=int)
grid = pv.UniformGrid()
grid.dimensions = resolution+1 # the grid points are the edges of the cells: add 1 for last edge
grid.spacing = 2/resolution # spacing of the grid
grid.origin = (-1,-1,-1) # point from which the grid grows
grid

UniformGrid,Information
N Cells,27000
N Points,29791
X Bounds,"-1.000e+00, 1.000e+00"
Y Bounds,"-1.000e+00, 1.000e+00"
Z Bounds,"-1.000e+00, 1.000e+00"
Dimensions,"31, 31, 31"
Spacing,"6.667e-02, 6.667e-02, 6.667e-02"
N Arrays,0


The cells of this grid will be the voxels, that will have cell value `1` if the mesh is occupying the voxel, and `0` otherwise. We will call this the `occupancy` field. In order to evaluate whether the surface is occupying the voxel, we will need to calculate the distance from the center of each cell to the surface of the mesh. Note that this operation is parallel-processing friendly and is much faster on a GPU.

In [4]:
# now we grab the cell centers
points = grid.cell_centers()

# and compute the distance form the mesh
points.compute_implicit_distance(mesh,inplace=True)

PolyData (0x7f4354a1abe0)
  N Cells:	27000
  N Points:	27000
  N Strips:	0
  X Bounds:	-9.667e-01, 9.667e-01
  Y Bounds:	-9.667e-01, 9.667e-01
  Z Bounds:	-9.667e-01, 9.667e-01
  N Arrays:	1

Now, in order to evaluate which voxels have the surface passing through them, we set a thresholding value for the distance between the center of the cells and the surface: for example, when $d<\sqrt{3}l$, where $l$ is the voxel linear size, the surface is intersecting the voxel, so we can consider the voxel occupied.

Now, the `grid` object has a data array associated with the cells, and we can export the field as a `np.array` to serve as an input for our model.

In [5]:
l = grid.spacing[0]
threshold = 3**(0.5)*l
mask = abs(points.point_data['implicit_distance']) < threshold

grid.cell_data['occupancy']=np.zeros(grid.n_cells,dtype=np.int8)
grid.cell_data['occupancy'][mask] = 1


grid

UniformGrid (0x7f435aefde80)
  N Cells:	27000
  N Points:	29791
  X Bounds:	-1.000e+00, 1.000e+00
  Y Bounds:	-1.000e+00, 1.000e+00
  Z Bounds:	-1.000e+00, 1.000e+00
  Dimensions:	31, 31, 31
  Spacing:	6.667e-02, 6.667e-02, 6.667e-02
  N Arrays:	1

In [6]:
#check to see if export works
array=grid.cell_data['occupancy'].reshape(resolution)
grid.cell_data['occupancy'] = array

pl = pv.Plotter()
pl.add_mesh(grid.threshold(0.5),color='red')
pl.view_vector([0.7,0.7,1.2])

pl.show()

Scene(background_color='#4c4c4c', camera={'position': [1.4418437437145084, 1.4418437437145084, 2.4717321320820…